In [2]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [16]:
data = np.load('us_topo.npy')
X, y = data[:,:-1], data[:,-1]

In [17]:
from dask.distributed import Client

In [18]:
state = np.random.get_state()

In [19]:
np.random.shuffle(X)

In [20]:
np.random.set_state(state)

In [21]:
np.random.shuffle(y)

In [22]:
X, y = X[:2000], y[:2000]

In [23]:
with open('GPs.pkl', 'rb') as file:
    from pickle import load
    hgdl_GPs = load(file)

/home/elliott/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator GaussianProcessRegressor from version 0.23.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [24]:
hgdl_GPs = hgdl_GPs[:10]

In [25]:
for j in range(4):
    print('j',j)
    mine = hgdl_GPs[j]
    print('\tLog Likelihood', mine.log_marginal_likelihood_value_.round(3))
    print('\tR^2',GaussianProcessRegressor(kernel=mine.kernel_, optimizer=None).fit(X,y).score(X,y).round(4))

    CV = []
    for i in range(3):
        test = X[i::3], y[i::3]
        train = np.append(X[(i+1)%3::3], X[(i+2)%3::3], 0), np.append(y[(i+1)%3::3], y[(i+2)%3::3])
        tmp = GaussianProcessRegressor(kernel=mine.kernel_, optimizer=None).fit(*train).score(*test)
        CV.append(tmp)


    print('\tCV',np.mean(CV).round(4))
    print()

j 0
	Log Likelihood -8624.881
	R^2 1.0
	CV 0.8973

j 1
	Log Likelihood -8762.799
	R^2 1.0
	CV 0.92

j 2
	Log Likelihood -9106.186
	R^2 1.0
	CV 0.9201

j 3
	Log Likelihood -10001.89
	R^2 1.0
	CV 0.6226

